## Installing MB-System on Ubuntu 14.04 and 16.04 LTS

#### Monica Schwehr
#### Monterey Bay Aquarium Research Institute  <mschwehr@mbari.org>


**MB-System** is an open source software package for the processing and display of bathymetry and backscatter imagery data derived from multibeam, interferometry, and sidescan sonars. 

The MB-System website http://www.mbari.org/products/research-software/mb-system/ has installation instructions for building MB-System on MacOS X and Linux. This document will offer a more detailed how-to guide for installing MB-System on Ubuntu 14.04 LTS and 16.04 LTS.


### Required Software

* GMT 5.2.1 or later
    * On Ubuntu 14.04LTS GMT 5.2.1 is included in the UbuntuGIS repository
        * instructions to add repository are below
    * On Ubuntu 16.04LTS, GMT 5.2.1 is included in the standard Ubuntu reposity
    * I'll also include directions on building from source
* GDAL 1.11 or later
* NetCDF 3.5.1
* Perl 5.x (this has not been tested with Perl 6.0)
* Perl Parallel-Forkmanager module
* X11
* Motif
* FFTW
* OTPSnc or OTPS2 (if you're going to be modeling tidal corrections)
* ghostview or gv
    * gv is actually a bit more powerful than ghostview and can handle poster-zised images

### UbuntuGIS Repository

The UbuntuGIS repository (https://wiki.ubuntu.com/UbuntuGIS) will bring in tools like GMT, GDAL, MB-System, QGIS, etc. but the versions of the software tools are often a couple iterations behind. 

As of 2/14/17 the version of GMT included in the default Ubuntu 14.04 software repository is 4.5.11 and the version in the UbuntuGIS repository is 5.1.2. Therefore, to get version 5.2.1+ you must install from source (instructions are below).

The UbuntuGIS repository is a great repo to add, as the software included is continuously being added to and updated. There is also an unstable version that has more recent versions of the some of the software tools, but is not thoroughly tested.

To add this repository to your system:
     * sudo add-apt-repository ppa:ubuntugis/ppa
     * sudo apt-get update
     * sudo apt-get upgrade

### Installing GMT 5.2.1 via repository

In Ubuntu 16.04LTS GMT 5.2.1 is included in the standard Ubuntu repository, no need to add the UbuntuGIS repo. so you can simply install it via apt-get

GMT will also install gmt-gshhg-full by default, I would recommend going with gmt-gshhg instead. This includes all resolutions and will automatically pick the best one based off the scale of your plot.

________________________________
sudo apt-get install gmt gmt-gshhg 
________________________________

### Installing GMT 5.2.1 from source
#### Required method if on a Ubuntu version below 16.04 LTS
If the UbuntuGIS repository does not have the version of GMT you want (perhaps you want to use 5.3 instead of 5.2.1) or you just want to install from source, you can follow the directions below:

#### Software Requirements
* Cmake 2.8.5 or later
* build-essential
* Proj4
    * only need development files, but I'd recommend the full package of cartographic projections
* NetCDF 4.0 or later
* GDAL
    * only need development files, but full prog is nice for translating data between different projection outside of GMT
* FFTW3
    * fast fourier transform library
* PCRE 
    * perl-compatible regular expressions
* LAPACK
    * linear algebra tool pack
    
    
*This list is not exhaustive, and you may need more or less programs and libraries depending on your current linux configuration and any previously installed software. Any missing tools will be identified during the build process*


** We can install these dependencies all in one go via the command:**
___________________________________________________________
    sudo apt-get install build-essential cmake libnetcdf-dev libgdal-dev gdal-bin libfftw3-dev libpcre3-dev 
    liblapack-dev libproj-dev proj-bin
___________________________________________________________ 

** Download GMT 5.2.1 (or later)**

1. Go to: http://gmt.soest.hawaii.edu/projects/gmt/wiki/Download
+ You'll need:
    + gmt-5.2.1-src.tar.gz
    + gshhg-gmt-2.3.6.tar.gz - Global Self-consistent Hierarchical High-resolution Geography
    + dcw-gmt-1.1.2.tar.gz - country polygons
+ cd to directory and before unpacking, verify downloand integrity with the checksum if you like
    + md5sum gmt-5.2.1-src.tar.gz
    + should return: df96d3cc9a93bc3c049f1523ada57117
+ tar -xf gmt-5.2.1-src.tar.gz
+ you'll also need to unpack the gshhg and dwg files somewhere. 
    + When installing via a package manager, GMT places them in /usr/share/
    + you may want to pick a location outside of /usr/ such as your home directory
+ cd into the unpacked gmt-5.2.1 directory
+ cd in cmake
+ cp ConfigUserTemplate.cmake ConfigUser.cmake
    + edit ConfigUser.cmake to tell GMT where to install and where the geography data is
        + set (CMAKE_INSTALL_PREFIX "path to install gmt")
            + do not edit this line if you want gmt to install into the default directory structure
                + On 14.04 I think this is /usr/bin/
        + set (GSHHG_ROOT "path to gshhg")
            + example: set (GSHHG_ROOT /usr/share/gmt-gshhg)
        + set (DCW_ROOT "path to dcw")
+ cd out of make, so you are in gmt-5.2.1:
+ run the following:
    + mkdir build
    + cd build
    + cmake ..
    + make
    + sudo make install
        + if you are building outside of /usr/ you can run this without the sudo command

** Remember: If you installed GMT outside of the default directory, you'll want to the add path to the bin directory to your path in your .bashrc or .bash_profile (e.g. export PATH=$PATH:path_to/gmt/bin)**





### Let's test our GMT install

----
    gmt pscoast -R-130/-30/-50/50 -Jm0.025i -B30g30:.Mercator: -Di -W > mercator.ps
    gv mercator.ps
----


When you run the above cell, you should end up with map centered on the equator, showing parts of North and South America

 <img src="mercator.png">

### The rest  of the MB-System requirements can be installed in one line


A lot of the dependencies of MB-System are shared by GMT 5.2, so all you should need to add is:
___________________    
    sudo apt-get install libmotif-dev libmotif-common xorg-dev libsdl1.2-dev libsdl-image1.2-dev libxp-dev
    libfftw3-3 libfftw3-dev netcdf-bin gv csh libgmt-dev libgmt5 libproj9 proj-bin gdal-bin libgdal-dev
    gfortran libparallel-forkmanager-perl
____________________


## Now, for MB-System itself:
	
These steps assume you have downloaded a tar.gz file from the website: http://www.mbari.org/products/research-software/mb-system/

The steps are the same whether you're on 14.04LTS or 16.04LTS

-----
     * unpack directory (tar -xf xxx.tar.gz), and cd into it
     * type ./configure at prompt
-----

If you are still missing any major packages, configure will let you know

If you want to install into a specific directory (I do this because I test multiple versions), add the prefix option: 

-----
    ./configure --prefix=$HOME/<name of directory>
    
    e.g.
    
    ./configure --prefix=$HOME/MBsystem-552282
-----

Run make
   make ensures it can find all the necessary libraries, so it is possible to get make errors complaining about missing libraries even if configure passes, most missing libraries can be installed with sudo apt-get install
`<name of library>`. 

Once make succeeds, you then run the actual install

-----
    make
    sudo make install
-----


## Now to set our GMT configure file

The main difference between GMT 4 and GMT 5 is that in version 5, a lot of the tools have been rewritten as modules. This is why in out GMT test above we had to type "gmt pscoast" instead of just the command "pscoast."

This affects the way some MB-System commands that include calls to GMT commands (such as mbcontour & mbswath) function. Therefore, we need to set up our gmt-config file.

The necessary MB-System library that we need to make GMT aware of (mbsystem.so) is in the MB-System install directory, under the lib subdirectory.

Run the command gmtset from your home directory to copy over the default gmt.conf file:

-----
    gmt gmtset  GMT_CUSTOM_LIBS   /path_to_MB-Sys_install_directory/lib/mbsystem.so 
-----

So for me, I installed MB-Sys in my home directory, not usr/local/ because I have multiple versions on my machine.
The above command for me would look like:


-----
    gmt gmtset  GMT_CUSTOM_LIBS   $HOME/MBsystem-552282/lib/mbsystem.so 
-----


### Lets Test the install

-----
    mbgrdviz
-----

Should bring up the MB-System grid visualization tool.
Click on Help --> About and you should see the version number


<img src="mbgrdviz_about.png">

## If you encounter problems

**1)** You get hung up processes running certain commands that automatically open ghostview or an image viewer, (e.g: mbm_plot runs, but running the resulting .cmd file hangs) make sure your mbdefaults are set for postscript viewer and screen image viewer:

------
    Set the postscript viewer using

        mbdefaults -Dgv -V

    Set your favorite screen image viewer (e.g. feh) using

        mbdefaults -Ifeh -V
-----

**2)** You get a shared library file error trying to run MB-Sys commands: "mbgrdviz: error while loading shared libraries: libmbio.so.0: cannot open shared object file: No such file or directory >" 

This can happen if you installed MB-Sys in a location other than the default (you used --prefix= when you ran ./configure)

libmbio.so should be located in your MB-Sys install directory, under the lib subdirectory 

To define the path to the shared libraries directory:

-----
    LD_LIBRARY_PATH=$HOME/MBsystem-552282/lib/ 
    export LD_LIBRARY_PATH
-----


### So now you should be good to go with MB-System on your Ubuntu system.

I'll make another post and upload some test data soon, so we can run through some of the basic MB-System commands and you can compare your results to mine to test for consistency. 